## Problem Statement
#### Compare return on investment for a rental property that is offered on the private market vs on AirBnb


## US Markets to Explore,
* San Francisco 
* New York
* Las Vegas
* Lake Tahoe
* Miami
* Los Angeles

### Property Type
The property must be an Apartment, House, Condominium, Townhouse, Cottage, Guesthouse, Hostel, Bungalow, Boutique hotel, Bed and breakfast, Hotel, Tiny house, Resort, Villa, Timeshare, Cabin, Castle, Earth house with a minimum of 1 bedroom and maximum of 7 bedrooms

### Calculation
After selecting the properties,calculate the rental rates of different areas based on the features to bedroom ratio from traditional rentals and Airbnb listings.

For the Airbnb occupancy rate, we check the number of bookings on a monthly basis, while the Airbnb occupancy rate displayed on the platform is based on the number of bookings that took place in the past 10 months. From there, we are then able to project the Airbnb occupancy rates for other properties based on the property’s rental rate and the property type. Finally, the estimated Airbnb rental income is calculated by multiplying the rental rate by the occupancy rate. 

Data Obtained from:

#### SF, NY
Murray Cox, 2018, “Detailed Listings data for San Francisco”, Inside Airbnb, http://insideairbnb.com/get-the-data.html

#### Explore the following Datasets

* AirBnb
* HomeAway
* Travelocity
* ListHub
* Rent Jungle
* Hotpads
* Zillow

## Ideas to Explore

### Features:
* Investiate Zillow Rental Prices vs Monthyl AirBnb Prices, example compare 1 bedrooms in a specific area
* Seasonal(Destination Type)
* Weather Patters(Climate)
* Walkability Index
* From Neighborhood look at zillow average home prices

In [1]:
#Import libraries
import pandas as pd
import numpy as np

import statsmodels.api as sm
import statsmodels.formula.api as smf
import patsy

import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import RidgeCV
%matplotlib inline

#Set display options
pd.set_option('display.max_columns', None)

C:\Users\moham\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
# Create initial DataFrame
df_sf = pd.DataFrame()
sf_df = pd.read_csv('..\\Data\\San Francisco Data\\listings.csv.gz',
                   error_bad_lines=False)
sf_df = df_sf.append(sf_df)

In [3]:
#Function to preprocess data
def preprocess(df):
    
    #Standardize column names
    df.columns = df.columns.str.strip()
    
    #Keep columns of interest
    df = df[['bathrooms','bedrooms','accommodates','price','monthly_price',\
             'property_type','zipcode','neighbourhood_cleansed','amenities','availability_365',\
             'number_of_reviews','review_scores_rating','review_scores_accuracy',\
             'review_scores_cleanliness','review_scores_checkin','review_scores_communication','review_scores_location',\
             'review_scores_value']]
    
    #Might include the following
    #'availability_30','availability_60',\'availability_90',
                              
    #Update property types
    #Modify listing names that might be considered a condo
    condo_list=['Guesthouse','Bungalow','Tiny house','Cottage']
    df = update_property_value('property_type',condo_list,'Condominium', df)
    
    #Modify listing names that might be considered an apartment
    apartment_list = ['Serviced apartment','Aparthotel','Timeshare']
    df = update_property_value('property_type',apartment_list,'Apartment', df)
    
    #Modify listing names that might be considered an apartment
    house_list = ['Hostel','Boutique hotel','Bed and breakfast','Villa','Cabin','Earth house']
    df = update_property_value('property_type',house_list,'House', df)
    
    #Keep properties that are an Apartment Townhouse Condominium or House
    df = df.loc[(df['property_type'] == 'Apartment') | (df['property_type'] == 'Townhouse') | (df['property_type'] == 'Condominium')\
                | (df['property_type'] == 'House')]
    
    #Update column names
    df.rename(columns={'neighbourhood_cleansed': 'neighborhood','price': 'nightly_rental_$','cleaning_fee':'cleaning_fee_$',\
                       'monthly_price':'monthly_rental_$'}, inplace=True)
    
    #Drop bedroom data with non values and convert to integer type
    df = df[np.isfinite(df['bedrooms'])]
    df['bedrooms']= df['bedrooms'].astype(int)
    
    #Drop bedroom data with non values and convert to integer type
    df = df[np.isfinite(df['bathrooms'])]
    
    #Drop rows with 0 bedrooms or bedrooms less than 7 bedrooms
    df = df[(df.bedrooms > 0) & (df.bedrooms <=4)]
    
    #Modify string dollar value of nightly price to float
    df['nightly_rental_$']=df['nightly_rental_$'].replace('[\$,]', '', regex=True).astype(float)
        
    #Update monthly price by multiplying each value of nightly_rental by 30
    df['monthly_rental_$']=df['nightly_rental_$']*30
      
    return df

def update_property_value(column_name,listing_list, updated_listing_name,df):
    for listing in listing_list:
        df[column_name].replace(listing, updated_listing_name, inplace=True)
    return df

In [4]:
df_sf = preprocess(sf_df)

C:\Users\moham\Anaconda3\lib\site-packages\pandas\core\generic.py:3924: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [5]:
#Determine number of listings per property_type
def number_of_listing(df):
    dicts = {}
    keys = df.property_type.unique()
    for i in keys:
        dicts[i] = len(df[df.property_type==str(i)])
    
    return dicts

sf_listing_dict = number_of_listing(df_sf)

In [6]:
print(sf_listing_dict)

{'Apartment': 2686, 'House': 2133, 'Condominium': 781, 'Townhouse': 137}


In [12]:
#Calculate average of reivew columns
#mean_availability_30 = df_sf['availability_30'].mean(skipna=True)
#mean_availability_60 = df_sf['availability_60'].mean(skipna=True)
#mean_availability_90 = df_sf['availability_90'].mean(skipna=True)
mean_availability_365 = df_sf['availability_365'].mean(skipna=True)
mean_num_reviews = df_sf['number_of_reviews'].mean(skipna=True)
mean_scores_rating = df_sf['review_scores_rating'].mean(skipna=True)
mean_score_accur = df_sf['review_scores_accuracy'].mean(skipna=True)
mean_score_clean = df_sf['review_scores_cleanliness'].mean(skipna=True)
mean_score_checkin = df_sf['review_scores_checkin'].mean(skipna=True)
mean_score_comm = df_sf['review_scores_communication'].mean(skipna=True)
mean_score_loc = df_sf['review_scores_location'].mean(skipna=True)
mean_review_score= df_sf['review_scores_value'].mean(skipna=True)

In [13]:
#Replace column NaN values with mean of column
#df_sf['availability_30'].fillna(int(round(mean_availability_30)), inplace=True)
#df_sf['availability_60'].fillna(int(round(mean_availability_60)), inplace=True)
#df_sf['availability_90'].fillna(int(round(mean_availability_90)), inplace=True)
df_sf['availability_365'].fillna(int(round(mean_availability_365)), inplace=True)

df_sf['number_of_reviews'].fillna(int(round(mean_num_reviews)), inplace=True)
df_sf['review_scores_rating'].fillna(int(round(mean_scores_rating)), inplace=True)
df_sf['review_scores_accuracy'].fillna(int(round(mean_score_accur)), inplace=True)
df_sf['review_scores_cleanliness'].fillna(int(round(mean_score_clean)), inplace=True)
df_sf['review_scores_checkin'].fillna(int(round(mean_score_checkin)), inplace=True)
df_sf['review_scores_communication'].fillna(int(round(mean_score_comm)), inplace=True)

df_sf['review_scores_location'].fillna(int(round(mean_score_loc)), inplace=True)
df_sf['review_scores_value'].fillna(int(round(mean_review_score)), inplace=True)

In [14]:
#Updated neighbourhood names in airbnb dataframe to match those from redfin
old_names = ['Ocean View ','Marina','Presidio','Downtown/Civic Center','Castro/Upper Market']
new_names = ['Ocean View Terrace','Marina District','Presidio Heights','Civic Center / Van Ness','Castro']

df_sf.replace('Ocean View', 'Ocean View Terrace', inplace=True)
df_sf.replace('Marina', 'Marina District',inplace=True)
df_sf.replace('Presidio', 'Presidio Heights',inplace=True)
df_sf.replace('Downtown/Civic Center', 'Civic Center / Van Ness',inplace=True)
df_sf.replace('Castro/Upper Market', 'Castro',inplace=True)


In [15]:
#Bring in zillow home value index and zillow rent index
xlsx_file_z_index = pd.ExcelFile('..\\Data\\San Francisco Data\\bedrooms_average.xlsx')
all_tier_z_index = pd.read_excel(xlsx_file_z_index, 'all_beds')


In [16]:
#Add Zillow Home Value and Rent Index Information for individual neighborhoods
df_sf = pd.merge(df_sf, all_tier_z_index, left_on=['neighborhood','bedrooms'], right_on=['neighborhood','bedrooms']);

In [17]:
#Round Zillow Indexes to nearest integer
df_sf['Zillow Home Value Index $']=df_sf['Zillow Home Value Index $'].astype(np.int64)
df_sf['Zillow Rent Index $']=df_sf['Zillow Rent Index $'].astype(np.int64)

# SF Walkability Info

In [18]:
#Bring in walkability and sales data from redfin
df_sf_info = pd.DataFrame()
sf_df_info = pd.read_csv('..\\Data\\San Francisco Data\\sf_walk_and_sale_df.csv')
sf_df_info = df_sf_info.append(sf_df_info)

In [19]:
#Clean additional Data Points
#Need to return to scrubbing function to address this issue
sf_df_info['walk_score_of_100'][2] = 81
sf_df_info['transit_score_of_100'][2] = 75
sf_df_info['bike_score_of_100'][2] = 71
sf_df_info['median_sale_price_$'][19] = '$1.25M'
sf_df_info['median_sale_price_$'][22] = '$1.01M'
sf_df_info['median_sale_price_$'][22] = '$1.01M'
sf_df_info['median_sale_price_$'][31] = '$761K'
sf_df_info['median_sale_price_$'][33] = '$1.35M'

C:\Users\moham\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\moham\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\Users\moham\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
C:\Users\moham\Anaconda3\l

In [20]:
#Convert Median Home Prices to Int Value
import re
def convert_string_to_int(df_column):
    for index, row in df_column.iteritems():
        if type(row) == float:
            pass
        elif (row[-1] == 'M') | (row[-1] == 'm'):
            number = re.findall("\d+\.\d+", row)
            df_column[index] = int(float(number[0])*1000000)
                      
        elif (row[-1] == 'K') | (row[-1] == 'k'):
            number = re.findall("\d+", row)
            df_column[index] = int(float(number[0])*1000)

    return df_column

sf_df_info['median_sale_price_$'] = convert_string_to_int(sf_df_info['median_sale_price_$'])

C:\Users\moham\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
C:\Users\moham\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


In [21]:
#Calculate average of sf_df_info columns
mean_walk_score_of_100 = sf_df_info['walk_score_of_100'].mean(skipna=True)
mean_transit_score_of_100 = sf_df_info['transit_score_of_100'].mean(skipna=True)
mean_bike_score_of_100 = sf_df_info['bike_score_of_100'].mean(skipna=True)
mean_median_sale_price = sf_df_info['median_sale_price_$'].mean(skipna=True)

In [22]:
#Replace column NaN values with mean of column
sf_df_info['walk_score_of_100'].fillna(int(round(mean_walk_score_of_100)), inplace=True)
sf_df_info['transit_score_of_100'].fillna(int(round(mean_transit_score_of_100)), inplace=True)
sf_df_info['bike_score_of_100'].fillna(int(round(mean_bike_score_of_100)), inplace=True)
sf_df_info['median_sale_price_$'].fillna(int(round(mean_median_sale_price)), inplace=True)

In [23]:
#Save Cleaned SF Info Data to CSV
#Save data to csv file
sf_df_info.to_csv('..\Data\San Francisco Data\cleaned_sf_walk_and_sale_df.csv', index=False)

In [24]:
#Add sf_df_info to df_sf dataframe, merging on neighborhood
df_sf = pd.merge(df_sf, sf_df_info, left_on=['neighborhood'], right_on=['neighborhood']);

# One Hot Encoding

In [25]:
#One Hot encode property type data and join to dataframe
df_sf = df_sf.join(pd.get_dummies(df_sf['property_type']));

In [26]:
#One Hot encode neighorhood data and join to dataframe
df_sf = df_sf.join(pd.get_dummies(df_sf['neighborhood']));

In [27]:
#Remove all non values from zipcode column
df_sf = df_sf[(df_sf['zipcode'].notnull())]

In [28]:
#Remove all 'CA' values from zipcode column
df_sf = df_sf[(df_sf['zipcode'] != 'CA')]

In [29]:
df_sf['zipcode'] = df_sf['zipcode'].apply(pd.to_numeric)

In [30]:
#One Hot encode zipcode data and join to dataframe
df_sf = df_sf.join(pd.get_dummies(df_sf['zipcode']));

In [31]:
#One Hot encode bathroom data and join to dataframe
#df_sf = df_sf.join(pd.get_dummies(df_sf['bathrooms']));

In [32]:
#One Hot encode bedroom data and join to dataframe
#df_sf = df_sf.join(pd.get_dummies(df_sf['bedrooms']));

In [33]:
df_sf.sample(3)

,bathrooms,bedrooms,accommodates,nightly_rental_$,monthly_rental_$,property_type,zipcode,neighborhood,amenities,availability_365,number_of_reviews,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,Zillow Home Value Index $,Zillow Rent Index $,walk_score_of_100,transit_score_of_100,bike_score_of_100,median_sale_price_$,Apartment,Condominium,House,Townhouse,Bayview,Bernal Heights,Castro,Chinatown,Civic Center / Van Ness,Crocker Amazon,Diamond Heights,Excelsior,Financial District,Glen Park,Golden Gate Park,Haight Ashbury,Inner Richmond,Inner Sunset,Lakeshore,Mission,Nob Hill,Noe Valley,North Beach,Ocean View Terrace,Outer Mission,Outer Richmond,Outer Sunset,Pacific Heights,Parkside,Potrero Hill,Presidio Heights,Russian Hill,Seacliff,South of Market,Twin Peaks,Visitacion Valley,West of Twin Peaks,Western Addition,94014,94080,94102,94103,94104,94105,94107,94108,94109,94110,94111,94112,94113,94114,94115,94116,94117,94118,94121,94122,94123,94124,94127,94131,94132,94133,94134,94158
4441,1.0,1,2,240.0,7200.0,Apartment,94111,North Beach,"{TV,""Cable TV"",Internet,Wifi,Kitchen,""Free par...",272,0,96.0,10.0,10.0,10.0,10.0,10.0,9.0,1246000,4811,99.0,95.0,73.0,1550000,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4090,1.0,1,5,195.0,5850.0,Apartment,94109,Russian Hill,"{TV,""Cable TV"",Internet,Wifi,Kitchen,Heating,""...",2,40,97.0,10.0,10.0,10.0,10.0,10.0,9.0,1658100,4043,96.0,93.0,67.0,1680000,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4743,2.0,3,6,425.0,12750.0,House,94121,Inner Richmond,"{TV,Wifi,Kitchen,""Free parking on premises"",""F...",0,3,93.0,10.0,10.0,9.0,10.0,10.0,10.0,2081400,5692,92.0,77.0,88.0,1320000,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0


# Keyword Extraction from Amenities

In [34]:
#Define amenities key words for three categories
basic_amenities_key_words = ['wifi','internet','fireplace','tv','iron','towels','bed','soap', \
                             'toilet' 'paper','heat','conditioning','hot water','shampoo','linens',\
                             'washer' 'dryer']

facilities_amenities_key_words = ['parking','backyard','garden','porch','tub','jacuzzi','pool','bbq','grill','patio',\
                                  'balcony','safety','detector','extinguisher','aid','garage','kid','parking']  

dining_amenities_key_words = ['kitchen','washer','coffee','microwave','refrigerator','oven','stove','dish','silver']                                             

In [35]:
def key_words_in_df_column(df_column,column_name,key_words_list):
    
    df = pd.DataFrame(columns=[column_name])
        
    for row in df_column:
        #Define pattern list and regex pattern
        pattern_list=[]
        pattern = re.compile('([^\s\w]|_)+')
    
        #List with key words extracted from key_words_list
        my_list=[]
    
        #Individual keywords list
        auxiliaryList = []
        
        #Split amenities listing by comma
        sentence_to_check = [x.strip() for x in row.lower().split(',')]
        
        #Strip keywords in amenities list of any non-alphanumerical characters
        for word in sentence_to_check:
            strippedList = pattern.sub('', word)
            pattern_list.append(strippedList)
        
        #Extract keywords and place in list
        for word in key_words_list:
            for words in pattern_list:
                if word in words: 
                    my_list.append(word)

        #Extract unique words from list
        for word in my_list:
            if word not in auxiliaryList:
                auxiliaryList.append(word)

        df.loc[-1] = len(auxiliaryList)  # adding a row
        df.index = df.index + 1  # shifting index
        df = df.sort_index()
        
    return(df)

In [36]:
df_basic_amenities = key_words_in_df_column(df_sf['amenities'],'length_basic_amenities',basic_amenities_key_words)
df_facilities_amenities = key_words_in_df_column(df_sf['amenities'],'length_facilities_amenities',facilities_amenities_key_words)
df_dining_amenities = key_words_in_df_column(df_sf['amenities'],'length_dining_amenities',dining_amenities_key_words)

In [37]:
#Print Stats of basic amenities
print("Min: ",df_basic_amenities['length_basic_amenities'].min())
print("Max: ",df_basic_amenities['length_basic_amenities'].max())
print("Mean: ",df_basic_amenities['length_basic_amenities'].mean())
print("Median: ",df_basic_amenities['length_basic_amenities'].median())
print("Mode: ",df_basic_amenities['length_basic_amenities'].mode())
print("Std: ",df_basic_amenities['length_basic_amenities'].std())

Min:  0
Max:  12
Mean:  6.818349299926308
Median:  7.0
Mode:  0    8
dtype: object
Std:  1.8822384343679548


In [38]:
#Cuttoff = At Least 6 Basic Amenities Average
len(df_basic_amenities[df_basic_amenities['length_basic_amenities']>=6]);

mask = df_basic_amenities.length_basic_amenities < 6
column_name = 'length_basic_amenities'
df_basic_amenities.loc[mask, column_name] = 0

mask = df_basic_amenities.length_basic_amenities >=6
column_name = 'length_basic_amenities'
df_basic_amenities.loc[mask, column_name] = 1


In [39]:
#Print Stats of facilities amenities
print("Min: ",df_facilities_amenities['length_facilities_amenities'].min())
print("Max: ",df_facilities_amenities['length_facilities_amenities'].max())
print("Mean: ",df_facilities_amenities['length_facilities_amenities'].mean())
print("Median: ",df_facilities_amenities['length_facilities_amenities'].median())
print("Mode: ",df_facilities_amenities['length_facilities_amenities'].mode())
print("Std: ",df_facilities_amenities['length_facilities_amenities'].std())

Min:  0
Max:  13
Mean:  4.794952100221076
Median:  4.0
Mode:  0    4
dtype: object
Std:  2.7132057566120116


In [40]:
#Cuttoff = At Least 4 Facility Amenities
len(df_facilities_amenities[df_facilities_amenities['length_facilities_amenities']>=4]);

mask = df_facilities_amenities.length_facilities_amenities < 4
column_name = 'length_facilities_amenities'
df_facilities_amenities.loc[mask, column_name] = 0

mask = df_facilities_amenities.length_facilities_amenities >=4
column_name = 'length_facilities_amenities'
df_facilities_amenities.loc[mask, column_name] = 1

In [41]:
#Print Stats of dining amenities
print("Min: ",df_dining_amenities['length_dining_amenities'].min())
print("Max: ",df_dining_amenities['length_dining_amenities'].max())
print("Mean: ",df_dining_amenities['length_dining_amenities'].mean())
print("Median: ",df_dining_amenities['length_dining_amenities'].median())
print("Mode: ",df_dining_amenities['length_dining_amenities'].mode())
print("Std: ",df_dining_amenities['length_dining_amenities'].std())

Min:  0
Max:  9
Mean:  4.505526897568165
Median:  2.0
Mode:  0    2
dtype: object
Std:  3.379907741232476


In [42]:
#Cuttoff = At Least 2 Dining Amenities
len(df_dining_amenities[df_dining_amenities['length_dining_amenities']>=2]);

mask = df_dining_amenities.length_dining_amenities < 4
column_name = 'length_dining_amenities'
df_dining_amenities.loc[mask, column_name] = 0

mask = df_dining_amenities.length_dining_amenities >=4
column_name = 'length_dining_amenities'
df_dining_amenities.loc[mask, column_name] = 1

In [43]:
df_dining_amenities.length_dining_amenities.unique()

array([0, 1], dtype=object)

# Merge Amenities to df_sf

In [44]:
#Reset Dataframe indices in df_sf 
df_sf = df_sf.reset_index(drop=True)

In [45]:
#Merge Basic Amenities Data 
df_sf['basic_amenities'] = df_basic_amenities['length_basic_amenities'].sort_values(ascending=False)

In [46]:
#Merge Facilities Amenities Data 
df_sf['facility_amenities'] = df_facilities_amenities['length_facilities_amenities'].sort_values(ascending=False)

In [47]:
#Merge Dining Amenities Data 
df_sf['dining_amenities'] = df_dining_amenities['length_dining_amenities'].sort_values(ascending=False)

# Master Cell

In [48]:
#Drop Property Type Column That's Been One Hot Encoded
df_sf.drop(['property_type'], axis=1, inplace=True)

In [49]:
#Drop Zipcode Column That's Been One Hot Encoded
df_sf.drop(['zipcode'], axis=1, inplace=True)

In [50]:
#Drop Neighborhood Column That's Been One Hot Encoded
df_sf.drop(['neighborhood'], axis=1, inplace=True)

In [51]:
#Drop Amenities Column That's Been One Hot Encoded
df_sf.drop(['amenities'], axis=1, inplace=True)

In [52]:
df_sf.sample(3)

,bathrooms,bedrooms,accommodates,nightly_rental_$,monthly_rental_$,availability_365,number_of_reviews,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,Zillow Home Value Index $,Zillow Rent Index $,walk_score_of_100,transit_score_of_100,bike_score_of_100,median_sale_price_$,Apartment,Condominium,House,Townhouse,Bayview,Bernal Heights,Castro,Chinatown,Civic Center / Van Ness,Crocker Amazon,Diamond Heights,Excelsior,Financial District,Glen Park,Golden Gate Park,Haight Ashbury,Inner Richmond,Inner Sunset,Lakeshore,Mission,Nob Hill,Noe Valley,North Beach,Ocean View Terrace,Outer Mission,Outer Richmond,Outer Sunset,Pacific Heights,Parkside,Potrero Hill,Presidio Heights,Russian Hill,Seacliff,South of Market,Twin Peaks,Visitacion Valley,West of Twin Peaks,Western Addition,94014,94080,94102,94103,94104,94105,94107,94108,94109,94110,94111,94112,94113,94114,94115,94116,94117,94118,94121,94122,94123,94124,94127,94131,94132,94133,94134,94158,basic_amenities,facility_amenities,dining_amenities
1274,1.0,1,2,148.0,4440.0,0,26,97.0,10.0,10.0,10.0,10.0,10.0,10.0,1473400,3848,85.0,78.0,70.0,1350000,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1
964,1.0,1,1,145.0,4350.0,20,6,97.0,10.0,10.0,10.0,10.0,10.0,9.0,1616300,5079,96.0,80.0,80.0,1350000,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0
1560,1.0,2,5,258.0,7740.0,0,3,100.0,10.0,9.0,10.0,10.0,10.0,9.0,1347200,4656,85.0,78.0,70.0,1350000,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [90]:
df_sf.to_csv('..\Data\San Francisco Data\cleaned_sf_data_with_info_and_amen.csv', index=False)

# Calculate ROI

Return ÷ Total Amount Paid Out of Pocket = ROI 

# Assumptions
- All Cash Purchase, no loan 
- Closing costs are calculate at 3.5 Percent of median sale price
- 10000 modeling costs
- 1% Rule on Maintenance - set 1% of home value toward maintenance
- Insurance and Tax Costs Aprox 2%
- San Francisco real estate appreciated 93.56% over the last ten years, which is an average annual home appreciation rate of 6.83% , Future Growth = (1+0.0683)^10 = 
- Buying and selling property after  15 years

In [ ]:
monetary_df = df_sf[['Zillow Rent Index $', 'Zillow Home Value Index $', 'median_sale_price_$','nightly_rental_$','monthly_rental_$','availability_365']].copy()

In [89]:
monetary_df.head(2)

,Zillow Rent Index $,Zillow Home Value Index $,median_sale_price_$,nightly_rental_$,monthly_rental_$,availability_365,yearly_rental_on_month_$,yearly_rental_on_avail_$,closing_costs_$,total_out_pocket_$,total_appreciation_$,yearly_rental_on_air_month_$,yearly_rental_on_zil_month_$,ROI_Air_Avail_%,ROI_Air_Best_%,ROI_Zillow_Best_%
0,3448,1053400,765000,170.0,5100.0,64,61200.0,511700.0,26775.0,352795.0,986108.867742,612000.0,413760,424,452,396
1,3448,1053400,765000,110.0,3300.0,59,39600.0,336600.0,26775.0,352795.0,986108.867742,396000.0,413760,374,391,396


In [88]:
#Set Yearly Rental Potential if Rented Monthly
monetary_df['yearly_rental_on_air_month_$'] = monetary_df['monthly_rental_$']*12*10

#Set Yearly Rental Based on Available Days
monetary_df['yearly_rental_on_avail_$'] = (365-monetary_df['availability_365'])*monetary_df['nightly_rental_$']*10

#Set Yearly Rental Based on Zillow Index
monetary_df['yearly_rental_on_zil_month_$'] = monetary_df['Zillow Rent Index $']*12*10

#Set Closing Costs
monetary_df['closing_costs_$'] = monetary_df['median_sale_price_$'] * .035

#Total_out_of_pocket_cost
monetary_df['total_out_pocket_$'] = monetary_df['closing_costs_$'] + 10000 + monetary_df['Zillow Home Value Index $']*0.01*10 \
                                    +monetary_df['Zillow Home Value Index $']*0.02*10

#Appreciation
monetary_df['total_appreciation_$'] = (1+0.0683)**10 * monetary_df['Zillow Home Value Index $'] \
                                        - monetary_df['Zillow Home Value Index $']

# Return ÷ Total Amount Paid Out of Pocket = ROI

#ROI Airbnb Availability
monetary_df['ROI_Air_Avail_%'] = ((monetary_df['yearly_rental_on_avail_$'] + monetary_df['total_appreciation_$']) / monetary_df['total_out_pocket_$'] *100).astype(np.int64)

#ROI Airbnb Best
monetary_df['ROI_Air_Best_%'] = ((monetary_df['yearly_rental_on_air_month_$'] + monetary_df['total_appreciation_$']) / monetary_df['total_out_pocket_$'] *100).astype(np.int64)

#ROI Airbnb Best
monetary_df['ROI_Zillow_Best_%'] = ((monetary_df['yearly_rental_on_zil_month_$'] + monetary_df['total_appreciation_$']) / monetary_df['total_out_pocket_$'] *100).astype(np.int64)

In [91]:
df_sf.to_csv('..\Data\San Francisco Data\montetary_sf_data.csv', index=False)

# Drop Categorical Data From Master Cell and Bring in Monetary ROI

In [92]:
df_sf['ROI'] = 

,bathrooms,bedrooms,accommodates,nightly_rental_$,monthly_rental_$,availability_365,number_of_reviews,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,Zillow Home Value Index $,Zillow Rent Index $,walk_score_of_100,transit_score_of_100,bike_score_of_100,median_sale_price_$,Apartment,Condominium,House,Townhouse,Bayview,Bernal Heights,Castro,Chinatown,Civic Center / Van Ness,Crocker Amazon,Diamond Heights,Excelsior,Financial District,Glen Park,Golden Gate Park,Haight Ashbury,Inner Richmond,Inner Sunset,Lakeshore,Mission,Nob Hill,Noe Valley,North Beach,Ocean View Terrace,Outer Mission,Outer Richmond,Outer Sunset,Pacific Heights,Parkside,Potrero Hill,Presidio Heights,Russian Hill,Seacliff,South of Market,Twin Peaks,Visitacion Valley,West of Twin Peaks,Western Addition,94014,94080,94102,94103,94104,94105,94107,94108,94109,94110,94111,94112,94113,94114,94115,94116,94117,94118,94121,94122,94123,94124,94127,94131,94132,94133,94134,94158,basic_amenities,facility_amenities,dining_amenities
0,1.0,1,3,170.0,5100.0,64,180,97.0,10.0,10.0,10.0,10.0,10.0,10.0,1053400,3448,96.0,89.0,89.0,765000,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1.0,1,2,110.0,3300.0,59,363,97.0,10.0,10.0,10.0,10.0,10.0,10.0,1053400,3448,96.0,89.0,89.0,765000,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,1


In [ ]:
Zillow Home Value Index $	Zillow Rent Index $	

# ROI Calculation Return on Investment

ROI = (potential_gain – cost_of_investment) / cost_of_investment

150,000 income property purchased with a 20 Percent down payment of 30,000. Let’s assume your mortgage of 120,000 is fixed for 30 years at a 4.38 percent interest rate.

The total return on investment (TROI) provides a better and more complete measure of a property’s financial performance. That is because it factors in amortization and appreciation gained over time.

Total ROI = (BTCF + Net Sales Proceeds – Initial Cash Investment) / Initial Cash Investment

#### Availability of your rental based on features

#### Availability of your rental out of 365 days * monthly price

#### ROI Calculation based on above income vs income if rented monthly




In [ ]:
#df_missing_zip_codes = df_sf.loc[df_sf['zipcode'].astype(str).str.contains('nan')]

In [ ]:
#List of neighborhoods with missing zipcodies
#df_missing_zip_codes = list(df_missing_zip_codes.groupby(['neighborhood','zipcode']).groups.keys())

In [ ]:
df_sf.to_csv('..\Data\San Francisco Data\cleaned_sf_data_with_info_and_amen.csv', index=False)

In [ ]:
#df_zip_code_list = list(df_sf.groupby(['neighborhood','zipcode']).groups.keys())

* Get Dataset, run.info, what type my data is, look at heat map, postively and negatively corelated things, pair plot, general
patterns, box and violen plots, categorical bar charts, Look at how patterns change

Create a hypothesis of what my linear model will tell me

Fit Basic Model, before feature extraction, Look at features, and evaluation 

Linear regression assumptions, residuals are normall distributed

Grid search and standard scalar

# Data Analysis

In [ ]:
df_sf.head(1)

In [ ]:
test

In [ ]:
df_sf_test=df_sf.copy()

In [ ]:
df_sf_test.drop(['neighborhood','access','amenities'], axis=1, inplace=True);

In [ ]:
df_sf_test.corr()

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))
sns.heatmap(df_sf_test.corr(), cmap="seismic", annot=True, vmin=-1, vmax=1, ax=ax);

In [ ]:
sns.pairplot(df_sf_test, aspect=1.5);

In [ ]:
columns = ['X0' , 'X1' , 'X2' , 'X3' ,'X4', 'X5' ,'X6' ,'X7' ,'X8' , 'X9' , 'X10' , 'X11' , 'X12' , 'X13' , 'X14' , 'X15' , 'X16' , 'X17' , 'X18' , 'X19' , 'X20' , 'X21' , 'X22' , 'X23' , 'X24' , 'X25' , 'X26' , 'X27' , 'X28' , 'X29' , 'X30' , 'X31' , 'X32' , 'X33' , 'X34' , 'X35' , 'X36' , 'X37' , 'X38' , 'X39' , 'X40' , 'X41' , 'X42' , 'X43' , 'X44' , 'X45' , 'X46' , 'X47' , 'X48', 'X49' , 'X50','X51']

In [ ]:
y,X = patsy.dmatrices('X4 ~ X0 + X1 + X2 + X5 + X9 + X10 + X11 + X12 + X13 + X14 + X15 + X16 + X17 + X18 + X19 + X20 + X21 + X22 + X23 + X24 + X25 + X26 + X27 + X28 + X29 + X30 + X31 + X32 + X33 + X34 + X35 + X36 + X37 + X38 + X39 + X40 + X41 + X42 + X43 + X44 + X45 + X46 + X47 + X48 + X49 + X50' , data=df_sf_test, return_type="dataframe")